In [4]:
import sys
!{sys.executable} -m pip install psycopg2-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 26.1 MB/s eta 0:00:0000:0100:01


In [19]:
import psycopg2

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    host="corso-kg-postgres",
    database="corso-kg-postgres",
    user="postgres",
    password="kgCefriel"
)

# Create a cursor object to interact with the database
cur = conn.cursor()

# Query to get the list of tables in the database
tables_query = """
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
    ORDER BY table_name
"""

# Execute the tables query
cur.execute(tables_query)
tables = cur.fetchall()

# Iterate over the tables
for table in tables:
    table_name = table[0]
    print("Table:", table_name)

    # Query to get 3 results for each table to check the columns
    columns_query = f"""
        SELECT *
        FROM {table_name}
        LIMIT 3
    """

    # Execute the columns query
    cur.execute(columns_query)
    columns = [desc[0] for desc in cur.description]

    print("Columns:")
    print(columns)

    print("")

Table: assignment
Columns:
['employee_id', 'competence_id', 'level', 'interest']

Table: competences
Columns:
['id', 'dimension', 'competence', 'description']

Table: employees
Columns:
['id', 'name', 'surname', 'unit']



In [30]:
# Custom query
custom_query = f"""
    WITH jane_competences AS (
        SELECT competence_id
        FROM employees
        JOIN assignment ON employees.id = assignment.employee_id
        WHERE employees.name = 'Jane' AND employees.surname = 'Wilson'
    )
    SELECT employees.id, employees.name, employees.surname, COUNT(assignment.competence_id) AS num_competences
    FROM employees
    JOIN assignment ON employees.id = assignment.employee_id
    WHERE assignment.competence_id IN (SELECT competence_id FROM jane_competences) 
        AND NOT(employees.name = 'Jane' AND employees.surname = 'Wilson')
    GROUP BY employees.id, employees.name, employees.surname
    ORDER BY num_competences DESC;
"""

# Execute the custom query
cur.execute(custom_query)
custom_results = cur.fetchall()
# Get the column names
columns = [desc[0] for desc in cur.description]

# Print the custom query results as a table
for col in columns:
    print(col, end="\t")
print()
for row in custom_results:
    for col in row:
        print(col, end="\t")
    print()

id	name	surname	num_competences	
EMPLOYEE_21	Emily	Brown	4	
EMPLOYEE_12	John	Johnson	4	
EMPLOYEE_2	Michael	Smith	4	
EMPLOYEE_20	Michael	Taylor	3	
EMPLOYEE_11	David	Lee	3	
EMPLOYEE_25	Olivia	Johnson	3	
EMPLOYEE_8	Michael	Brown	3	
EMPLOYEE_9	John	Smith	3	
EMPLOYEE_19	John	Johnson	3	
EMPLOYEE_17	Daniel	Davis	2	
EMPLOYEE_30	David	Wilson	2	
EMPLOYEE_18	Michael	Smith	2	
EMPLOYEE_23	Jane	Brown	2	
EMPLOYEE_29	Olivia	Miller	2	
EMPLOYEE_6	Emily	Davis	2	
EMPLOYEE_26	Michael	Taylor	2	
EMPLOYEE_13	John	Miller	2	
EMPLOYEE_15	David	Brown	2	
EMPLOYEE_7	Emma	Davis	1	
EMPLOYEE_14	Jane	Smith	1	
EMPLOYEE_28	Daniel	Wilson	1	
EMPLOYEE_4	John	Smith	1	
EMPLOYEE_1	Michael	Johnson	1	
EMPLOYEE_27	Emily	Miller	1	
EMPLOYEE_22	John	Johnson	1	
EMPLOYEE_24	David	Harris	1	
EMPLOYEE_3	Olivia	Clark	1	
EMPLOYEE_16	David	Smith	1	


In [ ]:
# Close the cursor and the connection
cur.close()
conn.close()